In [1]:
# !pip install python-dotenv
from dotenv import load_dotenv
import os
# os.getcwd()

load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
# 5.0 Memory
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory # save all conversation
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough


model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

# memory = ConversationBufferMemory()
# memory.save_context({"input": "Hi"}, {"output": "How are you?"})
# memory.load_memory_variables({})
memory = ConversationBufferMemory(return_messages=True) # 챗 형태 출력


def load_memory(_):
    x = memory.load_memory_variables({})
    return {"history": x["history"]}


chain = RunnablePassthrough.assign(history=load_memory) | prompt | model

inputs = {"message": "hi im bob"}
response = chain.invoke(inputs)
response

In [ ]:
# 5.1
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
) # window_size 4개의 대화만 저장하고 나머지는 삭제

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

In [ ]:
add_message(5, 5)

memory.load_memory_variables({}) # 1 삭제

In [ ]:
# 5.2
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# save only summary of conversation
memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
add_message("South Korea is so pretty", "I wish I could go!!!")
get_history()

In [2]:
# 5.3 summary + buffer
# 메모리의 메시지 수를 저장, 최근 정보만 추적
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150, # 최대 토큰 수
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [3]:
add_message("South Korea is so pretty", "I wish I could go!!!")
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!')]}

In [4]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [5]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [6]:
add_message("How far is England from Argentina?", "I don't know! Super far!")
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is England from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [7]:
add_message("How far is England from America?", "I don't know! Super far!")
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is England from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is England from America?'),
  AIMessage(content="I don't know! Super far!")]}

In [9]:
# 5.4 Knowledge graph
# 대화를 지식 그래프 형태로 저장
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [10]:
# specific message
memory.load_memory_variables({"input": "who is Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea.')]}

In [11]:
add_message("Nicolas likes kimchi", "Wow that is so cool!")
memory.load_memory_variables({"inputs": "what does nicolas like"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea. Nicolas likes kimchi.')]}

In [ ]:
# 토큰 수 저장 - Conversation Token Buffer
# entities를 저장하는 방식도 있음

In [ ]:
# 5.5 chain에 메모리 적용
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain # off-the-shelf chain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # max_token_limit=80,
    memory_key="chat_history", # 메모리 키 지정, 템플릿 내 변수명 동일
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    # prompt=PromptTemplate.from_template({"question"}),
    verbose=True, # Prompt 로그를 볼 수 있음
)

chain.predict(question="My name is Nico")

In [ ]:
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")
memory.load_memory_variables({}) # 요약 내용 들어가 있음 ~ 프롬프트로 써야함

In [ ]:
# 5.6 메모리 저장 방식 결정
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True, # 메시지 클래스로 바꾸어 줌
)

# 메시지 공간을 지정함
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")

In [ ]:
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")

In [ ]:
# 5.7 LCEL로 표현하는 방법 (custom chain)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]

# chain = prompt | llm
# chain.invoke

# load_memory 먼저 불러온 이후 history에 저장
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [ ]:
invoke_chain("My name is nico")

In [ ]:
invoke_chain("What is my name?")

In [ ]:
# 5.8 메모리 저장하고 다시 프롬프트에 넣어서 활용함
# LLM cahin, prompt template, chat prompt template, manual way
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


# 메모리 불러오기
def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    # result = chain.invoke({"question": question, "history": load_memory()})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [19]:
# Homework
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# llm = ChatOpenAI(temperature=0.1)
chat = ChatOpenAI(
    temperature = 0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)


examples = [
    {
        "question": "How can you express The Godfather in 3 imojis?",
        "answer": """
        🕴️🍷🔫
        """,
    },
    {
        "question": "How can you express Pulp Fiction in 3 imojis?",
        "answer": """
        🍔💉🕺
        """,
    },
    {
        "question": "How can you express Fight Club in 3 imojis?",
        "answer": """
        🧑👊🥊
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)


# final_prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are movie expert, you give short answers"),
#     example_prompt,
#     ("human", "How can you express {movie} in 3 imojis?")
# ])
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a Movie AI that express movie"),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    # print(result)

In [20]:
invoke_chain("How can you express Top Gun in 3 imojis?")
# invoke_chain("Top Gun")
# invoke_chain("How can you express The Godfather")

🛩️🕶️🔥

In [21]:
invoke_chain("What is the movie I ask?")

The movie is "Top Gun"